In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

C:\Users\Marat\AppData\Local\Temp\ipykernel_5132\3125191635.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [5]:
from sklearn import model_selection, preprocessing, linear_model
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [6]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [7]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [8]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
def custom_standardization(input_data):
    
    return input_data

vocab_size = 10000
seq_len = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Составим нейросеть.

Эксперимент показал, насколько важен слой эмбеддинга - просто на плотных слоях метрика accuracy была около 50%. Учитывая, что у нас практически нет дисбаланса классов - это просто случайное гадание. И сеть совершенно не обучалась и не сдвигалась с этих значений.

С слоем эмбеддинга количество весов существенно увеличилось, но сеть начала работать.

In [11]:
embedding_dim=200

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Flatten(),
    Dense(1000, activation='relu'),
    Dropout(.2),
    Dense(500, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 200)          2000000   
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 1000)              20001000  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                        

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5


C:\Users\Marat\AppData\Roaming\Python\Python39\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


10633/10633 [==============================] - 18410s 2s/step - loss: 0.3729 - accuracy: 0.8045 - val_loss: 0.3347 - val_accuracy: 0.8279
Epoch 2/5
 8636/10633 [=======================>......] - ETA: 53:06 - loss: 0.2991 - accuracy: 0.8507

Как видно, после первой же эпохи модель начинает переобучаться.

Причем дропаут не помогает избавиться от переобучения, а лишь замедляет темп переобучения.

Поэтому обучим модель еще раз на одной эпохе:

In [ ]:
model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Flatten(),
    Dense(1000, activation='relu'),
    Dropout(.2),
    Dense(500, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, validation_data=valid_data, epochs=1)

In [ ]:
pred = model.predict(x_test)
pred = np.where(pred >= 0.5, 1, 0)
print(classification_report(pred, y_test))

Как видно, сеть работает, но очень отстает от логистической регрессии по мешку слов. Видимо, сеть очень простая :)